In [36]:
import torch
import importlib
import homework4
import numpy as np


importlib.reload(homework4)

CNP = homework4.CNP

In [ ]:
def generate_samples(traj):   
    random_rows = traj[np.random.choice(traj.shape[0], size=4, replace=False)]
    
    random_rows = np.array(random_rows)
    
    tensor_1 = torch.tensor([random_rows[:3]], dtype=torch.float32) 
    target_x = np.concatenate((random_rows[3][:2], [random_rows[3][4]]))
    tensor_2 = torch.tensor([[target_x]], dtype=torch.float32) 
    tensor_3 = torch.tensor([[random_rows[3][2:4]]], dtype=torch.float32) 
    
    return tensor_1, tensor_2, tensor_3

def train():
    d_x = 3
    d_y = 2
    model = CNP([d_x,d_y], 128, 3)
    model.train()
    training_data = np.load("states.npy")
    traj_count = len(training_data)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_lst = []
    for i in range(1_000_000):
        traj = training_data[np.random.randint(traj_count)]
        traj = torch.tensor(traj, dtype=torch.float32)
        samples, target_x, target_y = generate_samples(traj)
        loss = model.nll_loss(samples,target_x, target_y)
        loss_lst.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print(f"Epoch {i} Loss: {loss.item()}")

    return loss_lst, model    
        



In [38]:
loss_lst, model = train() 
model.eval()
torch.save(model.state_dict(), "model.pth")

Epoch 0 Loss: 1.1970949172973633
Epoch 1000 Loss: -1.3166563510894775
Epoch 2000 Loss: -1.3478864431381226
Epoch 3000 Loss: -1.3492813110351562
Epoch 4000 Loss: -1.382008671760559
Epoch 5000 Loss: -1.3101987838745117
Epoch 6000 Loss: -1.357648491859436
Epoch 7000 Loss: -1.3714462518692017
Epoch 8000 Loss: -1.2652702331542969
Epoch 9000 Loss: -1.3643547296524048
Epoch 10000 Loss: -1.3774372339248657
Epoch 11000 Loss: -1.3141387701034546
Epoch 12000 Loss: -1.3771777153015137
Epoch 13000 Loss: -1.3810648918151855
Epoch 14000 Loss: -1.3705166578292847
Epoch 15000 Loss: -1.374531626701355
Epoch 16000 Loss: -1.373558759689331
Epoch 17000 Loss: -1.3812880516052246
Epoch 18000 Loss: -1.3771785497665405
Epoch 19000 Loss: -1.3766765594482422
Epoch 20000 Loss: -1.3822733163833618
Epoch 21000 Loss: -1.3796900510787964
Epoch 22000 Loss: -1.2547647953033447
Epoch 23000 Loss: -1.3798322677612305
Epoch 24000 Loss: -1.3820492029190063
Epoch 25000 Loss: -1.3834145069122314
Epoch 26000 Loss: -1.381591200